In [ ]:
#| default_exp mocking.objects

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from pymoq.mocking.functions import FunctionMock

from fastcore.basics import patch_to

In [ ]:
#| hide
from fastcore.test import test_fail
from pymoq.argument_validators import ArgumentFunctionValidator

# Mocking objects
> Mocking an object with all its (public) attributes and methods

## Public names

Since the main purpose of `pymoq` is to mock interfaces (aka protocols), we need a way to extract the public members of an interface. By convention, public members should not start with an underscore.

### Methods

All methods whose name doesn't start with an underscore should be included. The only public method in the following class is `get`.

In [ ]:
#| export
from typing import Protocol

In [ ]:
class IWeb(Protocol):    
    def get(self, suffix:str) -> str:
        ...
    
    def _internal_stuff(self) -> None:
        ...

All names are exposed through the `dir` method...

In [ ]:
', '.join(dir(IWeb))

'__abstractmethods__, __annotations__, __class__, __class_getitem__, __delattr__, __dict__, __dir__, __doc__, __eq__, __format__, __ge__, __getattribute__, __gt__, __hash__, __init__, __init_subclass__, __le__, __lt__, __module__, __ne__, __new__, __parameters__, __reduce__, __reduce_ex__, __repr__, __setattr__, __sizeof__, __slots__, __str__, __subclasshook__, __weakref__, _abc_impl, _internal_stuff, _is_protocol, _is_runtime_protocol, get'

... which can be filtered for names that don't start with an underscore:

In [ ]:
#| export
def _is_public_name(name: str):
    "Returns whether or not a (member) name is public"
    return not name.startswith('_')

def get_public_names(protocol: type) -> list[str]:
    "Returns all names that are considered public from the given class"
    names = [name for name in dir(protocol) if _is_public_name(name)]
    return names

In [ ]:
assert get_public_names(IWeb)==['get']

### Attributes

Attributes defined in protocol classes are not directly stored in the `dir` list. Instead, they are accessible in `__annotations__`. Note that [protocol variables have to be defined](https://peps.python.org/pep-0544/#protocol-members) at class level, not inside the `__init__` method.

In [ ]:
class IStore(Protocol):
    store_id: int
    name: str
    _internal_key: int
    
    def get(self, name:str) -> int:
        ...

In [ ]:
IStore.__annotations__

{'store_id': int, 'name': str, '_internal_key': int}

In [ ]:
#| export
def get_public_attributes(protocol: type(Protocol)) -> list[str]:
    "Return a list of all attributes of the given protocol that are considered public."
    attributes =  [name for name in protocol.__annotations__.keys() if _is_public_name(name)]
    return attributes

In [ ]:
get_public_attributes(IStore)

['store_id', 'name']

In [ ]:
assert 'store_id' in get_public_attributes(IStore)
assert 'name' in get_public_attributes(IStore)
assert not '_internal_key' in get_public_attributes(IStore)

## Construction from Protocol

### Dynamic attribute access

Dynamic attribute access is possible by overriding the [special method](https://docs.python.org/3/reference/datamodel.html?highlight=__getattr__#object.__getattr__) `__getattr__`. This method is called when a name is not found in the current instance of the class.

In [ ]:
class Outer:
    def __init__(self):
        self.valid = 2
        self.values = {'inner': 1}
        
    def __getattr__(self, name: str):
        print(f'Calling __getattr__("{name}")')
        if name in self.values:
            return self.values[name]
        raise AttributeError(f'Name {name} not found in values dictionary')
        
o = Outer()

Attribute is present in the class instance, so its accessed directly:

In [ ]:
print(o.valid)

2


Attribute is not present in the class instance, so `__getattr__` is called:

In [ ]:
print(o.inner)

Calling __getattr__("inner")
1


In [ ]:
try:
    o.invalid
except Exception as e:
    print(type(e), e)

Calling __getattr__("invalid")
<class 'AttributeError'> Name invalid not found in values dictionary


## The Mock object

The `Mock` object is the central class that the user of pymoq will interact with. It should be initialized with a protocol, setup function-mocks for all protocol-methods and handle the call redirection to the correct mock.

In [ ]:
#| export
class Mock:
    def __init__(self, protocol: type(Protocol)):
        self._protocol = protocol
        self._init_function_mocks(self._protocol)
        
    def _init_function_mocks(self, protocol: type(Protocol)):
        public_names = get_public_names(protocol)
        self._function_mocks = {
            name: FunctionMock(getattr(protocol, name))
            for name in public_names
        }
    
    def __str__(self):
        return f'Mock[{self._protocol.__name__}]'
    
    def __repr__(self): return str(self)

In [ ]:
mock = Mock(IWeb)
assert str(mock) == 'Mock[IWeb]'

assert list(mock._function_mocks.keys()) == ['get']

mock._function_mocks

{'get': <pymoq.mocking.functions.FunctionMock>}

When a function is called on a `Mock`, it should check whether that function is part of the underlyings protocol public interface. If not, throw an `AttributeError`. If yes, return the appropriate function mock.

In [ ]:
#| export
@patch_to(Mock)
def __getattr__(self, name: str) -> FunctionMock:
    if name not in self._function_mocks:
        raise AttributeError(f"Name {name} not found in {self}")
        
    return self._function_mocks[name]

In [ ]:
mock = Mock(IWeb)

test_fail(lambda: mock.not_a_name)
assert isinstance(mock.get, FunctionMock)

With this we can now build a working prototype of a mocked protocol.

In [ ]:
mock = Mock(IWeb)

mock.get.setup(
    ArgumentFunctionValidator(lambda a: isinstance(a, str), name='suffix', position=1)).returns(lambda self,suffix: f'suffix: {suffix}')

In [ ]:
assert mock.get('anyString') == 'suffix: anyString'
assert mock.get(1) is None
test_fail(lambda: mock.get())

# Build library

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()